In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
import seaborn as sns

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from bespoke.blaze_preapprovals import blaze_uw as blaze_uw

%load_ext autoreload
%autoreload 2

# ~ Part 1: Create UW data ~
This part is to read multiple queries to pull all the relevant data needed to perform underwriting.

## Gross Margin

In [3]:
blaze_gm_data = pd.read_sql_query(blaze_uw.create_blaze_gm_query('MA'),engine)
blaze_gm_data['year_month'] = pd.to_datetime(blaze_gm_data['year_month'])

In [4]:
# Check N of shops to underwrite
blaze_gm_data[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,6
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,10
2022-03-31,8
2022-04-30,10


In [5]:
# only underwrite those with more than 6 months of history data
blaze_gm_data_long = blaze_gm_data[blaze_gm_data['month_available'] >= 5]
blaze_gm_data_long[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,5
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,9
2022-03-31,8
2022-04-30,10


In [18]:
gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])

## Monthly GMV change

## state website published benchmark

In [15]:
# use own bm

rev_change_bm_with_weight = pd.DataFrame([numpy.nan,-0.07, 0.01, -0.07, 0.09,0.07,0.02,0.11,-0.01,-0.05,-0.01,0.07,-0.02,-0.07,-0.04])
rev_change_bm_with_weight['year_month'] = gm_distr_all['year_month'].values
rev_change_bm_with_weight['weight'] = [numpy.nan,0.04,0.08,0.12,0.2,0.28,0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
rev_change_bm_with_weight.columns = ['rev_change_bm','year_month','rev_change_weight']
rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
rev_change_bm_with_weight

,rev_change_bm,year_month,rev_change_weight
0,NaN,2021-08-31,NaN
1,-0.07,2021-09-30,0.04
2,0.01,2021-10-31,0.08
3,-0.07,2021-11-30,0.12
4,0.09,2021-12-31,0.20
5,0.07,2022-01-31,0.28
6,0.02,2022-02-28,0.36
7,0.11,2022-03-31,0.44
8,-0.01,2022-04-30,0.52
9,-0.05,2022-05-31,0.60


In [20]:

blaze_gm_data_long_with_rev  = pd.merge(blaze_gm_data_long,rev_change_bm_with_weight,on=['year_month'], how='inner')
blaze_gm_data_long_with_rev['rev_change_var'] = blaze_gm_data_long_with_rev['rev_change'] - blaze_gm_data_long_with_rev['rev_change_bm']
blaze_gm_data_long_with_rev['rev_change_points'] = [blaze_uw.gmv_change_variance_point_mapping(n) for n in blaze_gm_data_long_with_rev['rev_change_var']]
blaze_gm_data_long_with_rev['rev_change_total'] = blaze_gm_data_long_with_rev['rev_change_points'] * blaze_gm_data_long_with_rev['rev_change_weight']



In [21]:
blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['year_month']=='2022-10-31']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
69,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0
70,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0
71,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0
72,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0
73,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5
74,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0
75,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0
76,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0
77,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0
78,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0


## Inventory

In [22]:
#need to define the uw date first
uw_date = '2022-10-31'

In [23]:
inventory_date_list = pd.read_sql_query(blaze_uw.create_blaze_inventory_date_query('MA',uw_date),engine)

In [24]:
blaze_inv_data_all = pd.read_sql_query(blaze_uw.create_blaze_inventory_query('MA',tuple([str(x) for x in inventory_date_list['date']])),engine)
blaze_inv_data_all['year_month'] = pd.to_datetime(blaze_inv_data_all['year_month'])

In [25]:
#calculate fresh, stale, fresh + stale inventory
fresh_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'fresh'][['year_month','shopid','companyid','inventory']]
stale_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'stale'][['year_month','shopid','companyid','inventory']]
fresh_and_stale_inv = fresh_inv.merge(stale_inv, on = ['shopid','companyid','year_month'],how = 'outer')
fresh_and_stale_inv = fresh_and_stale_inv.fillna(0)
fresh_and_stale_inv.columns = ['year_month','shopid','companyid','inventory_fresh','inventory_stale']
fresh_and_stale_inv['inventory_fresh_and_stale'] = fresh_and_stale_inv['inventory_fresh']  + fresh_and_stale_inv['inventory_stale']

In [28]:
set(blaze_gm_data['shopid']) - set(fresh_and_stale_inv['shopid'])

{'605cef20813d4c08ebf50c06'}

In [27]:
#make sure we do not lack inventory data for too many locations (usually 1 or 2 are fine)
set(blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['year_month'] == uw_date]['shopid']) - set(fresh_and_stale_inv['shopid'])



set()

## Inventory Turnover

In [30]:
data_joined = pd.merge(blaze_gm_data_long_with_rev,fresh_and_stale_inv,on=['year_month','shopid','companyid'], how='inner')



In [31]:
data_joined['inventory_turnover'] = data_joined['sum_cogs_3m'] / data_joined['inventory_fresh_and_stale'] * 4
data_joined['year_month'] = pd.to_datetime(data_joined['year_month'])

In [32]:
# ##some sanity checks
# data_joined[data_joined['year_month'] >= '2021-10-31'].groupby(['year_month'])['inventory_turnover'].describe().reset_index()
# round(data_joined['inventory_turnover'].quantile(0.95),2)

17.02

# ~ Part 2: Start UW ~
Now that we have all the essential data for UW. We choose the date we want to perform underwriting on and start calculating the KPIs.

In [41]:
recent = data_joined[data_joined['year_month'] == '2022-10-31']
recent = recent.reset_index(drop = True)

In [42]:
# check final # of locations to UW
recent[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-10-31,10


## 1. gm % score

In [43]:
gm_threshold = [[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7]]

In [44]:
gm_score_data = recent.apply(lambda row: blaze_uw.get_gm_perc_scores(gm_threshold,row['avg_margin_3m'],row['avg_margin_6m'],row['avg_margin_all']),axis = 1)
recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_3m_score', 'gm_6m_score','gm_all_score'])],axis=1)



In [45]:

recent_with_gm[recent_with_gm['shopid'] == '5e7e9d34f795d008e7136fe1']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score
2,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796,177139.623167,17.865675,5,5,5


## 2. inventory turnover score

In [46]:
recent_with_gm['turnover_score'] = [10 if (recent_with_gm['inventory_turnover'][i] >= 6 and recent_with_gm['inventory_turnover'][i] <= 26) else 0 for i in range(len(recent_with_gm))]



## 3. rev change score

In [47]:
rev_change_score = blaze_gm_data_long_with_rev[['shopid','rev_change_total']].groupby('shopid').sum().reset_index()

In [48]:
rev_change_score['rev_change_score'] = [max(min(round(rev_change_score['rev_change_total'][i],2),10),-10) for i in range(len(rev_change_score))]
#rev_change_score.sort_values(by = 'rev_change_total')


## 4. gm $ score

In [50]:
gm_dollar_score = blaze_gm_data_long_with_rev[['shopid','profit_pre_tax_post_discount']].groupby('shopid').mean().reset_index()

In [51]:
gm_dollar_score['gm_dollar_score'] = [15 if gm_dollar_score['profit_pre_tax_post_discount'][i] >= 200000 else 0 for i in range(len(rev_change_score))]


In [52]:
recent_with_gm_and_rev_change = pd.merge(recent_with_gm,rev_change_score[['shopid','rev_change_score']],on=['shopid'], how='inner')
#recent_with_gm_and_rev_change

# ~ Part 3: Final scoring and credit limit & rate calculation ~

In [53]:
final = pd.merge(recent_with_gm_and_rev_change,gm_dollar_score[['shopid','gm_dollar_score']],on=['shopid'], how='inner')


In [56]:
final['total_gm_perc_score'] = final['gm_3m_score'] + final['gm_6m_score'] + final['gm_all_score']
final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
final['credit_limit_raw'] = [round(min(final['sum_cogs_3m'][i],final['inventory_fresh'][i]),0) for i in range(len(final))]
final['potential_credit_limit'] = round(final['credit_limit_raw'],-4) 
final['initial_credit_limit'] = final['potential_credit_limit'] / 2                         



In [57]:
final

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0,0,8,8,28.0,117398.0,120000.0,60000.0
1,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0,0,15,15,35.0,235230.0,240000.0,120000.0
2,2022-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,604509.33,636730.75,239024.96,365484.37,0.604597,15,791179.75,0.594942,0.582123,0.554735,-0.050604,-0.04,1.0,-0.010604,0.0,0.0,155181.827167,21957.796000,177139.623167,17.865675,5,5,5,10,-1.4,15,15,15,23.6,155182.0,160000.0,80000.0
3,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0,0,10,10,30.0,232326.0,230000.0,115000.0
4,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0,15,15,15,35.0,355654.0,360000.0,180000.0
5,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0,0,10,10,30.0,180317.0,180000.0,90000.0
6,2022-10-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,398911.22,345868.26,227042.19,171869.03,0.430845,11,547276.57,0.454801,0.431704,0.432142,0.153362,-0.04,1.0,0.193362,10.0,10.0,269629.819667,17477.120000,287106.939667,7.624707,-2,-2,-2,10,10.0,0,-6,-6,14.0,269630.0,270000.0,135000.0
7,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0,0,10,10,30.0,251272.0,250000.0,125000.0
8,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0,15,10,15,35.0,200066.0,200000.0,100000.0
9,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0,0,15,15,35.0,236329.0,240000.0,120000.0


In [58]:
# look at distribution of total score
final['total_score'].describe()

count    10.000000
mean     29.560000
std       6.658528
min      14.000000
25%      28.500000
50%      30.000000
75%      35.000000
max      35.000000
Name: total_score, dtype: float64

# ~ Part 4: Generate pre-approvals ~
Passing criteria:
1. total score >= 30
2. GM dollar >= 25K
3. Potential credit limit >= 30K

In [59]:
# per passing criterial above
final_pass = final[(final['total_score'] >= 24)&\
                  (final['profit_pre_tax_post_discount'] >= 25000)&\
                  (final['potential_credit_limit'] >= 30000)]
final_pass = final_pass.reset_index(drop=True)

In [60]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0,0,8,8,28.0,117398.0,120000.0,60000.0
1,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0,0,15,15,35.0,235230.0,240000.0,120000.0
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0,0,10,10,30.0,232326.0,230000.0,115000.0
3,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0,15,15,15,35.0,355654.0,360000.0,180000.0
4,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0,0,10,10,30.0,180317.0,180000.0,90000.0
5,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0,0,10,10,30.0,251272.0,250000.0,125000.0
6,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0,15,10,15,35.0,200066.0,200000.0,100000.0
7,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0,0,15,15,35.0,236329.0,240000.0,120000.0


In [61]:
final_pass['annual_rate'] = [blaze_uw.calculate_interest_rate(final_pass['total_score'][i],45)[1] for i in range(len(final_pass))]
final_pass['monthly_rate'] = [blaze_uw.calculate_interest_rate(final_pass['total_score'][i],45)[0] for i in range(len(final_pass))]


In [62]:
# look at the approval rate this month
final_pass.shape[0] / final.shape[0]

0.8

In [63]:
final_pass.shape[0]

8

In [64]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit,annual_rate,monthly_rate
0,2022-10-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,160258.92,77877.82,88243.34,0.531199,8,223910.36,0.536044,0.518130,0.420323,0.036580,-0.04,1.0,0.076580,10.0,10.0,117397.566000,6585.712000,123983.278000,7.223889,5,5,-2,10,10.0,0,8,8,28.0,117398.0,120000.0,60000.0,0.214,0.0178
1,2022-10-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,366812.95,168858.98,211206.13,0.555710,15,502076.65,0.548800,0.537614,0.525731,0.036128,-0.04,1.0,0.076128,10.0,10.0,235229.861000,8187.696000,243417.557000,8.250459,5,5,5,10,10.0,0,15,15,35.0,235230.0,240000.0,120000.0,0.200,0.0167
2,2022-10-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,365122.15,161323.42,175887.81,0.521595,15,512114.76,0.517596,0.502646,0.491483,-0.076443,-0.04,1.0,-0.036443,0.0,0.0,232325.513219,14232.266000,246557.779219,8.308231,5,5,0,10,10.0,0,10,10,30.0,232326.0,230000.0,115000.0,0.210,0.0175
3,2022-10-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,1043956.68,437667.52,503458.10,0.534953,15,1435174.27,0.531181,0.519522,0.502901,-0.098501,-0.04,1.0,-0.058501,-2.5,-2.5,355653.853000,3510.068000,359163.921000,15.983502,5,5,5,10,10.0,15,15,15,35.0,355654.0,360000.0,180000.0,0.200,0.0167
4,2022-10-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,218596.06,105636.53,132821.20,0.557001,7,288509.10,0.554925,0.544958,0.494061,0.090860,-0.04,1.0,0.130860,10.0,10.0,180317.482333,3912.789000,184230.271333,6.264098,5,5,0,10,10.0,0,10,10,30.0,180317.0,180000.0,90000.0,0.210,0.0175
5,2022-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,500332.58,248635.27,281790.24,0.531253,13,719671.18,0.512478,0.502954,0.495582,0.060146,-0.04,1.0,0.100146,10.0,10.0,251272.116333,12493.324333,263765.440667,10.913806,5,5,0,10,10.0,0,10,10,30.0,251272.0,250000.0,125000.0,0.210,0.0175
6,2022-10-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,508372.13,251999.86,265496.52,0.513040,15,755322.35,0.501870,0.504403,0.496247,0.017948,-0.04,1.0,0.057948,10.0,10.0,200065.910333,5959.202667,206025.113000,14.664666,5,5,0,10,10.0,15,10,15,35.0,200066.0,200000.0,100000.0,0.200,0.0167
7,2022-10-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,462616.26,194619.58,226440.81,0.537787,11,625605.73,0.530389,0.524699,0.528190,-0.089828,-0.04,1.0,-0.049828,0.0,0.0,236328.913000,39072.750000,275401.663000,9.086448,5,5,5,10,10.0,0,15,15,35.0,236329.0,240000.0,120000.0,0.200,0.0167


# ~ Part 5: output data ~

In [65]:
# # # only take the essential columns to output to csv file
# final_pass[['shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs',
#        'profit_pre_tax_post_discount', 'margin_pre_tax_post_discount',
#        'sum_cogs_3m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all',
#        'inventory_fresh_and_stale', 'inventory_turnover', 'gm_3m_score',
#        'gm_6m_score', 'gm_all_score', 'total_gm_perc_score', 'gm_dollar_score', 'total_gm_score', 'turnover_score', 'rev_change_score',
# 'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit','annual_rate',
#        'monthly_rate']]

,shopid,companyid,rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,sum_cogs_3m,avg_margin_3m,avg_margin_6m,avg_margin_all,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,total_gm_perc_score,gm_dollar_score,total_gm_score,turnover_score,rev_change_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit,annual_rate,monthly_rate
0,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,166121.16,77877.82,88243.34,0.531199,223910.36,0.536044,0.518130,0.420323,123983.278000,7.223889,5,5,-2,8,0,8,10,10.0,28.0,117398.0,120000.0,60000.0,0.214,0.0178
1,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,380065.11,168858.98,211206.13,0.555710,502076.65,0.548800,0.537614,0.525731,243417.557000,8.250459,5,5,5,15,0,15,10,10.0,35.0,235230.0,240000.0,120000.0,0.200,0.0167
2,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,337211.23,161323.42,175887.81,0.521595,512114.76,0.517596,0.502646,0.491483,246557.779219,8.308231,5,5,0,10,0,10,10,10.0,30.0,232326.0,230000.0,115000.0,0.210,0.0175
3,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,941125.62,437667.52,503458.10,0.534953,1435174.27,0.531181,0.519522,0.502901,359163.921000,15.983502,5,5,5,15,15,15,10,10.0,35.0,355654.0,360000.0,180000.0,0.200,0.0167
4,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,238457.73,105636.53,132821.20,0.557001,288509.10,0.554925,0.544958,0.494061,184230.271333,6.264098,5,5,0,10,0,10,10,10.0,30.0,180317.0,180000.0,90000.0,0.210,0.0175
5,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,530425.51,248635.27,281790.24,0.531253,719671.18,0.512478,0.502954,0.495582,263765.440667,10.913806,5,5,0,10,0,10,10,10.0,30.0,251272.0,250000.0,125000.0,0.210,0.0175
6,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,517496.38,251999.86,265496.52,0.513040,755322.35,0.501870,0.504403,0.496247,206025.113000,14.664666,5,5,0,10,15,15,10,10.0,35.0,200066.0,200000.0,100000.0,0.200,0.0167
7,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,421060.39,194619.58,226440.81,0.537787,625605.73,0.530389,0.524699,0.528190,275401.663000,9.086448,5,5,5,15,0,15,10,10.0,35.0,236329.0,240000.0,120000.0,0.200,0.0167


## Output the pre-approval list to a csv file
Always name the csv file as "blaze_ma_MONTH_approval.csv"

In [ ]:
# final_pass[['shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs',
#        'profit_pre_tax_post_discount', 'margin_pre_tax_post_discount',
#        'sum_cogs_3m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all',
#        'inventory_fresh_and_stale', 'inventory_turnover', 'gm_3m_score',
#        'gm_6m_score', 'gm_all_score', 'total_gm_perc_score', 'gm_dollar_score', 'total_gm_score', 'turnover_score', 'rev_change_score',
# 'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit','annual_rate',
#        'monthly_rate']].to_csv('blaze_ma_nov_approval.csv')

## Output all uw data to a csv file
Always name the csv file as "blaze_ma_MONTH_all.csv"

In [ ]:
# final[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
#        'margin_pre_tax_post_discount', 'sum_cogs_3m',
#        'avg_margin_2m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
#        'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
#        'total_gm_score','turnover_score', 'rev_change_score',  'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit']].to_csv('blaze_ma_nov_all.csv')


# ~ Part 6: Diff analysis to compare with last month ~

In [68]:
# read last month's files
#prev_approve = pd.read_csv('blaze_ma_oct_approval.csv',index_col=0)
prev_all = pd.read_csv('blaze_ma_oct_all.csv',index_col=0)

In [69]:
prev_all

,year_month,shopid,companyid,rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,total_gm_perc_score,gm_dollar_score,total_gm_score,turnover_score,rev_change_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,218091.65,244524.61,0.528569,645830.08,0.526690,0.525532,0.521119,0.527231,344517.360000,7.498375,5,5,5,15,0,15,10,10.0,35.0,296943.0,300000.0,150000.0
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,250567.88,249764.70,0.499197,688131.56,0.503090,0.503835,0.494262,0.492609,285163.659000,9.652444,5,0,0,5,0,5,10,10.0,25.0,279636.0,280000.0,140000.0
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,258661.77,378068.98,0.593766,884323.87,0.590115,0.586285,0.573676,0.551173,248252.414387,14.248786,5,5,5,15,15,15,10,10.0,35.0,215927.0,220000.0,110000.0
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,97528.40,121067.66,0.553842,261667.02,0.553887,0.548914,0.483571,0.483571,178099.270333,5.876880,5,-2,-2,1,0,1,0,10.0,11.0,178099.0,180000.0,90000.0
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,167578.21,199234.74,0.543151,503396.02,0.545345,0.542059,0.534984,0.523590,239122.137167,8.420735,5,5,5,15,0,15,10,10.0,35.0,227861.0,230000.0,115000.0
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,178329.67,186792.48,0.511589,535724.43,0.515596,0.504899,0.498357,0.489332,272400.058324,7.866730,5,0,-2,3,0,3,10,10.0,23.0,260210.0,260000.0,130000.0
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,492541.26,551415.42,0.528198,1505519.11,0.529294,0.523984,0.514009,0.500611,363158.299333,16.582511,5,5,5,15,15,15,10,10.0,35.0,360785.0,360000.0,180000.0
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,74694.29,85564.63,0.533915,219712.83,0.538467,0.534984,0.339953,0.404484,102272.556000,8.593227,5,-2,-2,1,0,1,10,10.0,21.0,99838.0,100000.0,50000.0
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,258151.46,250220.67,0.492200,746144.68,0.496285,0.500204,0.502461,0.495048,268164.234762,11.129667,5,5,0,10,15,15,10,10.0,35.0,260825.0,260000.0,130000.0
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,175702.28,170165.98,0.491997,437506.65,0.466779,0.455398,0.428353,0.432272,281156.240333,6.224392,-2,-2,-2,-6,0,-6,10,10.0,14.0,250532.0,250000.0,125000.0


In [ ]:
# newly approved this month
newly_added_list = list(set(final_pass['shopid']) - set(prev_approve['shopid']))
final_pass[final_pass['shopid'].isin(newly_added_list)][['shopid','total_score']].sort_values(by = 'shopid')

In [ ]:
# how they scored last month?
prev_all[prev_all['shopid'].isin(newly_added_list)][['shopid','total_score']].sort_values(by = 'shopid')

In [ ]:
# dropoffs
dropoff_lsit = list(set(prev_approve['shopid']) - set(final_pass['shopid']))

In [ ]:
final[final['shopid'].isin(dropoff_lsit)]

# ~ Part 7: Summary stats ~

In [66]:
print('Total number of shop to UW: ' + str(len(final))+ ' (have at least 6m of data)')
print('Total number of shops approved: ' + str(len(final_pass)))
print('The approval rate is: ' + str(round(len(final_pass)/len(final),2)))
print('Total Final Limit rounded: '+ str(sum(final_pass['potential_credit_limit'])))
print('Average Monthly Rate: ' + str(numpy.mean(final_pass['monthly_rate'])))

Total number of shop to UW: 10 (have at least 6m of data)
Total number of shops approved: 8
The approval rate is: 0.8
Total Final Limit rounded: 1820000.0
Average Monthly Rate: 0.0171375


# ~ scatch, please ignore ~

In [ ]:
# gm_distr_6m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_6m'].describe().reset_index()
# gm_distr_6m.index = pd.to_datetime(gm_distr_6m['year_month'])
# gm_distr_6m
# round(gm_distr_6m[['25%','50%','75%']].mean(),2)
# round(blaze_gm_data_long['avg_margin_6m'].quantile(0.95),2)

In [ ]:
# gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
# gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])
# round(gm_distr_all,3)
# round(gm_distr_all[['25%','50%','75%']].mean(),2)
# round(blaze_gm_data_long['avg_margin_all'].quantile(0.95),2)